<a href="https://colab.research.google.com/github/louisowen6/NLP_Stacking_Ensemble/blob/master/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount the Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Import Packages

In [0]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import gensim 
import nltk
from time import time
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics.pairwise import cosine_similarity

# Run it to obtain reproducible results across machines (from keras.io)
from __future__ import print_function
import numpy as np
import tensorflow as tf
import random as rn
import os
from tensorflow.keras import optimizers,backend as K
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
rn.seed(12345)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
tf.set_random_seed(1234)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

#For Neural Network
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate, Reshape, Dropout, Activation,Embedding,Flatten

# Import Supporting Files

In [0]:
#Google Word2Vec Pretrained Model
link ='https://drive.google.com/open?id=16A169DxZ-h9qU0i6rXCoSh_djkBWnd9V'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('GoogleNews-vectors-negative300.bin')   
model_w2v = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True) 

#GloVe Pretrained Model
link ='https://drive.google.com/open?id=1p1IN9O_fpSQzPTFB5Y8CCOMXPIPU1WGV'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('glove.twitter.27B.200d.txt')   
def loadGloveModel(gloveFilePath):
    print("Loading Glove Model")
    f = open(gloveFilePath,'r',encoding='utf-8')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model
model_GloVe=loadGloveModel('glove.twitter.27B.200d.txt')

16A169DxZ-h9qU0i6rXCoSh_djkBWnd9V
1p1IN9O_fpSQzPTFB5Y8CCOMXPIPU1WGV
Loading Glove Model
Done. 1193514  words loaded!


# Import Data

In [0]:
link ='https://drive.google.com/open?id=19nc79hUNZP0xH2IssgSQQTWxNjqAQZnT'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('df_train_final.csv')  
df_train = pd.read_csv('df_train_final.csv')
df_train=df_train.drop(['Unnamed: 0','index'],1)

link ='https://drive.google.com/open?id=1cDKGfay0oLkGQM0jz4nyk6c9CoE2YIpi'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('df_test_final.csv')  
df_test = pd.read_csv('df_test_final.csv')
df_test=df_test.drop(['Unnamed: 0','index'],1)

19nc79hUNZP0xH2IssgSQQTWxNjqAQZnT
1cDKGfay0oLkGQM0jz4nyk6c9CoE2YIpi


# Function Definition

In [0]:
def grid_model(df,hidden_nodes_1,hidden_nodes_2,hidden_nodes_3,input_dropout,hidden_dropout,activation_list,optimizer,epochs,batch_size=32,two_layer=False,sentence_vector=False):
    '''
    Functio for hyperparameter tuning
    '''
    #Initiate StratifiedKFold Class
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=500)
    #Split data into dependent and independent variable
    X=df.drop(['sentiment score'],1)
    y=df['sentiment score']
    #Prepare stratify variable
    stratify=df['source']
    #begin hyperparameter tuning looping
    for act in activation_list:
      cvscores = [] #list containing metrics score for each fold
      start=time()
      for train,val in kfold.split(X,stratify):
        X_train=X.loc[train,:]
        y_train=y[train]
        X_val=X.loc[val,:]
        y_val=y[val]

        if not sentence_vector:
          X_train=X_train.drop(['cashtag','spans','text','clean_text','base_text','source'],1)
          X_val=X_val.drop(['cashtag','spans','text','clean_text','base_text','source'],1)
        else:
          X_train=X_train.drop('source',1)
          X_val=X_val.drop('source',1)

        #Define MLP
        model = Sequential()
        model.add(Dropout(input_dropout, input_shape=(X_train.shape[1],),seed=0))
        model.add(Dense(hidden_nodes_1))
        model.add(Activation(act))
        model.add(Dropout(hidden_dropout))
        model.add(Dense(hidden_nodes_2))
        
        if not two_layer:
          model.add(Activation(act))
          model.add(Dropout(hidden_dropout))
          model.add(Dense(hidden_nodes_3))
          model.add(Activation('tanh'))
        else:
          model.add(Activation('tanh'))
  
        model.add(Dense(1))

        model.compile(loss='MSE',optimizer=optimizer,metrics=['cosine_proximity']) 
        model.fit(X_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=False,
                  validation_data=(X_val, y_val))
        score, cos = model.evaluate(X_val, y_val, verbose=False)
        cvscores.append(cos)
      end=time()
      print('Time Elapsed: {}'.format(end-start))
      print('Epochs: {}'.format(epochs))
      print('Hidden Nodes 1: {}'.format(hidden_nodes_1))
      print('Hidden Nodes 2: {}'.format(hidden_nodes_2))
      print('Hidden Nodes 3: {}'.format(hidden_nodes_3))
      print('Input Dropout Rate: {}'.format(input_dropout))
      print('Hidden Dropout Rate: {}'.format(hidden_dropout))
      print('Activation Function: {}'.format(act))
      print('Optimizer: {}'.format(optimizer))
      print('Stratified 10-Fold Cosine Similarity Avg Score: {}, Std Score: {}'.format(np.mean(cvscores),np.std(cvscores)))
      print('')
  
def model(df,df_test,hidden_nodes_1,hidden_nodes_2,hidden_nodes_3,input_dropout,hidden_dropout,act,optimizer,epochs,batch_size=32,two_layer=False,sentence_vector=False):
    '''
    Function for prediction
    '''
    if not sentence_vector:
      X_train=df.drop(['cashtag','spans','text','clean_text','base_text','source','sentiment score'],1)
      y_train=df['sentiment score']
      X_test=df_test.drop(['cashtag','spans','text','clean_text','base_text','source','sentiment score'],1)
      y_test=df_test['sentiment score']
    else:
      X_train=df.drop(['source','sentiment score'],1)
      y_train=df['sentiment score']
      X_test=df_test.drop(['source','sentiment score'],1)
      y_test=df_test['sentiment score']
    
    start=time()
    model = Sequential()
    model.add(Dropout(input_dropout, input_shape=(X_train.shape[1],),seed=0))
    model.add(Dense(hidden_nodes_1))
    model.add(Activation(act))
    model.add(Dropout(hidden_dropout))
    model.add(Dense(hidden_nodes_2))
        
    if not two_layer:
      model.add(Activation(act))
      model.add(Dropout(hidden_dropout))
      model.add(Dense(hidden_nodes_3))
      model.add(Activation('tanh'))
    else:
      model.add(Activation('tanh'))
  
    model.add(Dense(1))
    model.compile(loss='MSE',optimizer=optimizer,metrics=['cosine_proximity'])

    model.fit(X_train, y_train,
              batch_size=32,
              epochs=epochs,
              verbose=False)
    pred_test=model.predict(X_test, batch_size=batch_size)
    pred_test=pred_test.reshape(1,-1)
    end=time()
    print('Time Elapsed: {}'.format(end-start))
    print('Epochs: {}'.format(epochs))
    print('Hidden Nodes 1: {}'.format(hidden_nodes_1))
    print('Hidden Nodes 2: {}'.format(hidden_nodes_2))
    print('Hidden Nodes 3: {}'.format(hidden_nodes_3))
    print('Input Dropout Rate: {}'.format(input_dropout))
    print('Hidden Dropout Rate: {}'.format(hidden_dropout))
    print('Activation Function: {}'.format(act))
    print('Optimizer: {}'.format(optimizer))
    print('Test Cosine Similarity Score:{}'.format(cosine_similarity(pred_test,y_test.values.reshape(1,-1))[0][0]))
    print('')
    return(pd.Series(pred_test[0]))    

def tokenize(sentence):
  '''
  tokenize input into token
  '''
  token_list=nltk.regexp_tokenize(sentence, pattern=r"\s|[\.,;]\D", gaps=True)
  return(token_list)

def min_multiple_list(S):
  '''
  Minimum Pooling
  '''
    it=range(len(S)-1)
    minim=S[0]
    for i in it:
        minim=np.minimum(minim,S[i])
    return(minim)

def max_multiple_list(S):
  '''
  Maximum Pooling
  '''
    it=range(len(S)-1)
    maxim=S[0]
    for i in it:
        maxim=np.maximum(maxim,S[i])
    return(maxim)
  
def W2V_sentence_embedding(df,lite=False,span=False):
  '''
  return dataframe for W2V sentence embedding
  '''
  if not span:
    column='clean_text'
  else:
    column='spans'
    
  zero=np.array([float(0) for i in range(300)])
  nan=np.array([np.nan for i in range(300)])
  vec_W2V=lambda x: model_w2v[x] if x in model_w2v else zero
  W2V_avg=df[column].apply(lambda sent: pd.Series(tokenize(sent)).apply(lambda x: vec_W2V(x)).mean() if type(sent)==str else nan)
  
  if not lite:
    W2V_min=df[column].apply(lambda sent: min_multiple_list(pd.Series(tokenize(sent)).apply(lambda x: vec_W2V(x))) if type(sent)==str else nan)
    W2V_max=df[column].apply(lambda sent: max_multiple_list(pd.Series(tokenize(sent)).apply(lambda x: vec_W2V(x))) if type(sent)==str else nan)
    W2V_concat=W2V_avg.copy()
    it=range(len(W2V_concat))
    for i in it:
      W2V_concat[i]=(pd.Series(W2V_avg[i]).append(pd.Series(W2V_min[i]),ignore_index=True).append(pd.Series(W2V_max[i]),ignore_index=True)).values
    
    W2V_df=pd.DataFrame(dict(zip(W2V_concat.index, W2V_concat.values))).T
  else:
    W2V_df=pd.DataFrame(dict(zip(W2V_avg.index, W2V_avg.values))).T
    
  W2V_df['source']=df['source']
  W2V_df['sentiment score']=df['sentiment score']
  col=['x'+str(i) for i in W2V_df.drop(['source','sentiment score'],1).columns.tolist()]
  col.append('source')
  col.append('sentiment score')
  W2V_df.columns=col
  return(W2V_df)

def GloVe_sentence_embedding(df,lite=False,span=False):
  '''
  return dataframe for GloVe sentence embedding
  '''
  if not span:
    column='clean_text'
  else:
    column='spans'
    
  zero=np.array([float(0) for i in range(200)])
  nan=np.array([np.nan for i in range(200)])
  vec_GloVe=lambda x: model_GloVe[x] if x in model_GloVe else zero
  GloVe_avg=df[column].apply(lambda sent: pd.Series(tokenize(sent)).apply(lambda x: vec_GloVe(x)).mean() if type(sent)==str else nan)
  if not lite:
    GloVe_min=df[column].apply(lambda sent: min_multiple_list(pd.Series(tokenize(sent)).apply(lambda x: vec_GloVe(x))) if type(sent)==str else nan)
    GloVe_max=df[column].apply(lambda sent: max_multiple_list(pd.Series(tokenize(sent)).apply(lambda x: vec_GloVe(x))) if type(sent)==str else nan)
    GloVe_concat=GloVe_avg.copy()
    it=range(len(GloVe_concat))
    for i in it:
      GloVe_concat[i]=(pd.Series(GloVe_avg[i]).append(pd.Series(GloVe_min[i]),ignore_index=True).append(pd.Series(GloVe_max[i]),ignore_index=True)).values
    
    GloVe_df=pd.DataFrame(dict(zip(GloVe_concat.index, GloVe_concat.values))).T
  else:
    GloVe_df=pd.DataFrame(dict(zip(GloVe_avg.index, GloVe_avg.values))).T
    
  GloVe_df['source']=df['source']
  GloVe_df['sentiment score']=df['sentiment score']
  col=['x'+str(i) for i in GloVe_df.drop(['source','sentiment score'],1).columns.tolist()]
  col.append('source')
  col.append('sentiment score')
  GloVe_df.columns=col
  return(GloVe_df)

In [0]:
df_train.head()

# MLP Feature Driven

## Hyperparameter Tuning

In [0]:
grid_model(df_train,hidden_nodes_1=50,hidden_nodes_2=30,hidden_nodes_3=15,input_dropout=0.15,hidden_dropout=0.2,activation_list=['relu','tanh','elu'],optimizer='adam',epochs=200,two_layer=False,sentence_vector=False)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Time Elapsed: 226.1183626651764
Epochs: 200
Hidden Nodes 1: 50
Hidden Nodes 2: 30
Hidden Nodes 3: 15
Input Dropout Rate: 0.15
Hidden Dropout Rate: 0.2
Activation Function: relu
Optimizer: adam
Stratified k-Fold Cosine Similarity Avg Score: 0.8951565027236938, Std Score: 0.025400707498192787

Time Elapsed: 247.98554801940918
Epochs: 200
Hidden Nodes 1: 50
Hidden Nodes 2: 30
Hidden Nodes 3: 15
Input Dropout Rate: 0.15
Hidden Dropout Rate: 0.2
Activation Function: tanh
Optimizer: adam
Stratified k-Fold Cosine Similarity Avg Score: 0.9003740549087524, Std Score: 0.031864020973443985

Time Elapsed: 273.68214440345764
Epochs: 200
Hidden Nodes 1: 50
Hidden Nodes 2: 30
Hidden Nodes 3: 15
Input Dropout Rate: 0.15
Hidden Dropout Rate: 0.2
Activation Function: elu
Optimizer: adam
Stratified k-Fold Cosine Similarity Avg Score: 0.8882153630256653, Std Score: 0.026528913527727127



In [0]:
grid_model(df_train,hidden_nodes_1=50,hidden_nodes_2=30,hidden_nodes_3=15,input_dropout=0.1,hidden_dropout=0.2,activation_list=['tanh'],optimizer='adam',epochs=200,two_layer=False,sentence_vector=False)

Time Elapsed: 333.4433732032776
Epochs: 200
Hidden Nodes 1: 50
Hidden Nodes 2: 30
Hidden Nodes 3: 15
Input Dropout Rate: 0.1
Hidden Dropout Rate: 0.2
Activation Function: tanh
Optimizer: adam
Stratified k-Fold Cosine Similarity Avg Score: 0.8883042335510254, Std Score: 0.03374252840876579



## Prediction

In [0]:
pred_MLP_Feature_Driven=model(df_train,df_test,hidden_nodes_1=50,hidden_nodes_2=30,hidden_nodes_3=15,input_dropout=0.15,hidden_dropout=0.2,act='tanh',optimizer='adam',epochs=200,two_layer=False,sentence_vector=False)

Time Elapsed: 23.391836404800415
Epochs: 200
Hidden Nodes 1: 50
Hidden Nodes 2: 30
Hidden Nodes 3: 15
Input Dropout Rate: 0.15
Hidden Dropout Rate: 0.2
Activation Function: tanh
Optimizer: adam
Test Cosine Similarity Score:0.8780225047313124



In [0]:
pd.DataFrame(pred_MLP_Feature_Driven,columns=['pred_MLP_Feature_Driven']).to_csv('pred_MLP_Feature_Driven.csv')

# MLP Sentence Vector

In [0]:
#Word2Vec Sentence Embedding dataframe
W2V_df_train=W2V_sentence_embedding(df_train)
W2V_df_test=W2V_sentence_embedding(df_test)
#GloVe Sentence Embedding dataframe
GloVe_df_train=GloVe_sentence_embedding(df_train)
GloVe_df_test=GloVe_sentence_embedding(df_test)

## W2V Hyperparameter Tuning

In [0]:
grid_model(W2V_df_train,hidden_nodes_1=50,hidden_nodes_2=25,hidden_nodes_3=10,input_dropout=0.2,hidden_dropout=0.2,activation_list=['elu','tanh','relu'],optimizer='adam',epochs=200,batch_size=32,two_layer=False,sentence_vector=True)

Time Elapsed: 249.4553472995758
Epochs: 200
Hidden Nodes 1: 50
Hidden Nodes 2: 25
Hidden Nodes 3: 10
Input Dropout Rate: 0.2
Hidden Dropout Rate: 0.2
Activation Function: elu
Optimizer: adam
Stratified 10-Fold Cosine Similarity Avg Score: 0.5274509787559509, Std Score: 0.08557087182998657

Time Elapsed: 265.29081177711487
Epochs: 200
Hidden Nodes 1: 50
Hidden Nodes 2: 25
Hidden Nodes 3: 10
Input Dropout Rate: 0.2
Hidden Dropout Rate: 0.2
Activation Function: tanh
Optimizer: adam
Stratified 10-Fold Cosine Similarity Avg Score: 0.512036144733429, Std Score: 0.09375698119401932

Time Elapsed: 304.1298086643219
Epochs: 200
Hidden Nodes 1: 50
Hidden Nodes 2: 25
Hidden Nodes 3: 10
Input Dropout Rate: 0.2
Hidden Dropout Rate: 0.2
Activation Function: relu
Optimizer: adam
Stratified 10-Fold Cosine Similarity Avg Score: 0.5496419072151184, Std Score: 0.07226593047380447



## W2V Prediction

In [0]:
pred_MLP_W2V_Sentence_Vector=model(W2V_df_train,W2V_df_test,hidden_nodes_1=50,hidden_nodes_2=25,hidden_nodes_3=10,input_dropout=0.2,hidden_dropout=0.2,act='relu',optimizer='adam',epochs=200,two_layer=False,sentence_vector=True)

Time Elapsed: 32.39320516586304
Epochs: 200
Hidden Nodes 1: 50
Hidden Nodes 2: 25
Hidden Nodes 3: 10
Input Dropout Rate: 0.2
Hidden Dropout Rate: 0.2
Activation Function: relu
Optimizer: adam
Test Cosine Similarity Score:0.7065173223386277



In [0]:
pd.DataFrame(pred_MLP_W2V_Sentence_Vector,columns=['pred_MLP_W2V_Sentence_Vector']).to_csv('pred_MLP_W2V_Sentence_Vector.csv')

## GloVe Hyperparameter Tuning

In [0]:
grid_model(GloVe_df_train,hidden_nodes_1=90,hidden_nodes_2=50,hidden_nodes_3=30,input_dropout=0.2,hidden_dropout=0.35,activation_list=['relu','tanh','elu'],optimizer='adam',epochs=75,batch_size=32,two_layer=False,sentence_vector=True)

Time Elapsed: 314.93492913246155
Epochs: 75
Hidden Nodes 1: 90
Hidden Nodes 2: 50
Hidden Nodes 3: 30
Input Dropout Rate: 0.2
Hidden Dropout Rate: 0.35
Activation Function: relu
Optimizer: adam
Stratified 10-Fold Cosine Similarity Avg Score: 0.41555365920066833, Std Score: 0.08183988183736801

Time Elapsed: 337.6875786781311
Epochs: 75
Hidden Nodes 1: 90
Hidden Nodes 2: 50
Hidden Nodes 3: 30
Input Dropout Rate: 0.2
Hidden Dropout Rate: 0.35
Activation Function: tanh
Optimizer: adam
Stratified 10-Fold Cosine Similarity Avg Score: 0.34526166319847107, Std Score: 0.11206923425197601

Time Elapsed: 365.846720457077
Epochs: 75
Hidden Nodes 1: 90
Hidden Nodes 2: 50
Hidden Nodes 3: 30
Input Dropout Rate: 0.2
Hidden Dropout Rate: 0.35
Activation Function: elu
Optimizer: adam
Stratified 10-Fold Cosine Similarity Avg Score: 0.41400742530822754, Std Score: 0.08110479265451431



In [0]:
grid_model(GloVe_df_train,hidden_nodes_1=90,hidden_nodes_2=50,hidden_nodes_3=30,input_dropout=0.2,hidden_dropout=0.35,activation_list=['relu'],optimizer='sgd',epochs=75,batch_size=32,two_layer=False,sentence_vector=True)

Time Elapsed: 388.64111828804016
Epochs: 75
Hidden Nodes 1: 90
Hidden Nodes 2: 50
Hidden Nodes 3: 30
Input Dropout Rate: 0.2
Hidden Dropout Rate: 0.35
Activation Function: relu
Optimizer: sgd
Stratified 10-Fold Cosine Similarity Avg Score: 0.3071209788322449, Std Score: 0.09145727008581161



## GloVe Prediction

In [0]:
pred_MLP_GloVe_Sentence_Vector=model(GloVe_df_train,GloVe_df_test,hidden_nodes_1=90,hidden_nodes_2=50,hidden_nodes_3=30,input_dropout=0.2,hidden_dropout=0.35,act='relu',optimizer='adam',epochs=75,batch_size=32,two_layer=False,sentence_vector=True)

Time Elapsed: 40.818522930145264
Epochs: 75
Hidden Nodes 1: 90
Hidden Nodes 2: 50
Hidden Nodes 3: 30
Input Dropout Rate: 0.2
Hidden Dropout Rate: 0.35
Activation Function: relu
Optimizer: adam
Test Cosine Similarity Score:0.5177242936099722



In [0]:
pd.DataFrame(pred_MLP_GloVe_Sentence_Vector,columns=['pred_MLP_GloVe_Sentence_Vector']).to_csv('pred_MLP_GloVe_Sentence_Vector.csv')

# MLP Sentence Vector Lite

In [0]:
#Word2Vec Sentence Embedding dataframe
W2V_df_train_lite=W2V_sentence_embedding(df_train,lite=True)
W2V_df_test_lite=W2V_sentence_embedding(df_test,lite=True)
#GloVe Sentence Embedding dataframe
GloVe_df_train_lite=GloVe_sentence_embedding(df_train,lite=True)
GloVe_df_test_lite=GloVe_sentence_embedding(df_test,lite=True)

## W2V Lite Hyperparameter Tuning

In [0]:
grid_model(W2V_df_train_lite,W2V_df_test_lite,hidden_layer_nodes_list=[20,30,40],activation_list=['tanh','relu','elu'],optimizer='adam',two_layer=True,sentence_vector=True,epochs=150)

Hidden Layer Nodes: 20
Activation Function: tanh
Optimizer: adam
Test Cosine Similarity Score:0.6160594250686096
Time Elapsed: 88.42888355255127


Hidden Layer Nodes: 30
Activation Function: tanh
Optimizer: adam
Test Cosine Similarity Score:0.6306608461248566
Time Elapsed: 87.18056321144104


Hidden Layer Nodes: 40
Activation Function: tanh
Optimizer: adam
Test Cosine Similarity Score:0.6116119445716933
Time Elapsed: 87.4682195186615


Hidden Layer Nodes: 20
Activation Function: relu
Optimizer: adam
Test Cosine Similarity Score:0.6216701113948772
Time Elapsed: 89.16574192047119


Hidden Layer Nodes: 30
Activation Function: relu
Optimizer: adam
Test Cosine Similarity Score:0.6632882388736354
Time Elapsed: 88.30841946601868


Hidden Layer Nodes: 40
Activation Function: relu
Optimizer: adam
Test Cosine Similarity Score:0.625196227631343
Time Elapsed: 91.32043528556824


Hidden Layer Nodes: 20
Activation Function: elu
Optimizer: adam
Test Cosine Similarity Score:0.5993873969437827
Time Ela

In [0]:
grid_model(W2V_df_train_lite,W2V_df_test_lite,hidden_layer_nodes_list=[30],activation_list=['relu'],optimizer='sgd',two_layer=True,sentence_vector=True,epochs=150)

Hidden Layer Nodes: 30
Activation Function: relu
Optimizer: sgd
Test Cosine Similarity Score:0.5651049885677355
Time Elapsed: 91.97556614875793




## W2V Lite Prediction

In [0]:
pred_MLP_W2V_lite_Sentence_Vector=model(W2V_df_train_lite,W2V_df_test_lite,hidden=30,act='relu',optimizer='adam',two_layer=True,sentence_vector=True,epochs=150)

Hidden Layer Nodes: 30
Activation Function: relu
Optimizer: adam
Test Cosine Similarity Score:0.6572309135824692
Time Elapsed: 47.66274285316467




In [0]:
pd.DataFrame(pred_MLP_W2V_lite_Sentence_Vector,columns=['pred_MLP_W2V_lite_Sentence_Vector']).to_csv('pred_MLP_W2V_lite_Sentence_Vector.csv')

## GloVe Lite Hyperparameter Tuning

In [0]:
grid_model(GloVe_df_train_lite,GloVe_df_test_lite,hidden_layer_nodes_list=[25,30,40,50],activation_list=['tanh','relu','elu'],optimizer='adam',two_layer=True,sentence_vector=True,epochs=200)

Hidden Layer Nodes: 25
Activation Function: tanh
Optimizer: adam
Test Cosine Similarity Score:0.47636942290126194
Time Elapsed: 72.429368019104


Hidden Layer Nodes: 30
Activation Function: tanh
Optimizer: adam
Test Cosine Similarity Score:0.44591472742141924
Time Elapsed: 78.77393698692322


Hidden Layer Nodes: 40
Activation Function: tanh
Optimizer: adam
Test Cosine Similarity Score:0.40425984107559065
Time Elapsed: 77.10419225692749


Hidden Layer Nodes: 50
Activation Function: tanh
Optimizer: adam
Test Cosine Similarity Score:0.4657886758307373
Time Elapsed: 77.46132326126099


Hidden Layer Nodes: 25
Activation Function: relu
Optimizer: adam
Test Cosine Similarity Score:0.47663081534880636
Time Elapsed: 76.20769715309143


Hidden Layer Nodes: 30
Activation Function: relu
Optimizer: adam
Test Cosine Similarity Score:0.47497150800798105
Time Elapsed: 79.45700764656067


Hidden Layer Nodes: 40
Activation Function: relu
Optimizer: adam
Test Cosine Similarity Score:0.5100643990694882
Ti

In [0]:
grid_model(GloVe_df_train_lite,GloVe_df_test_lite,hidden_layer_nodes_list=[30],activation_list=['elu'],optimizer='sgd',two_layer=True,sentence_vector=True,epochs=200)

Hidden Layer Nodes: 30
Activation Function: elu
Optimizer: sgd
Test Cosine Similarity Score:0.39846414944963854
Time Elapsed: 72.12655472755432




## GloVe Lite Prediction

In [0]:
pred_MLP_GloVe_Lite_Sentence_Vector=model(GloVe_df_train_lite,GloVe_df_test_lite,hidden=30,act='elu',optimizer='adam',two_layer=True,sentence_vector=True,epochs=200)

In [0]:
pd.DataFrame(pred_MLP_GloVe_Lite_Sentence_Vector,columns=['pred_MLP_GloVe_Lite_Sentence_Vector']).to_csv('pred_MLP_GloVe_Lite_Sentence_Vector.csv')